In [0]:
#config rápida
account_name = "datalakeprojectrebrickab"
container_name = "silver"
account_key = ""

mount_point = f"/mnt/{container_name}"

if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
        source = f"wasbs://{container_name}@{account_name}.blob.core.windows.net",
        mount_point = mount_point,
        extra_configs = {f"fs.azure.account.key.{account_name}.blob.core.windows.net": account_key}
    )
    print("Montaje realizado exitosamente")
else:
    print(f"El punto {mount_point} ya está montado")

El punto /mnt/silver ya está montado


#### Ingesta de tablas a utilizar

In [0]:
list_names =[]
for f in dbutils.fs.ls("/mnt/silver/rebrickable/LEGO_Catalog_Database_cleaned/"):
    list_names.append(f.name)

list_names1 =[]
for f in dbutils.fs.ls("/mnt/silver/rebrickable/oltp_db_cleaned/"):
    list_names1.append(f.name)

In [0]:
for f in list_names:

    delta_path = f"/mnt/silver/rebrickable/LEGO_Catalog_Database_cleaned/{f}"

    delta_df = spark.read.format("delta").load(delta_path)

    delta_df.createOrReplaceTempView(f"{f.rstrip('/')}")

for f in list_names1:

    delta_path = f"/mnt/silver/rebrickable/oltp_db_cleaned/{f}"

    delta_df = spark.read.format("delta").load(delta_path)

    delta_df.createOrReplaceTempView(f"{f.rstrip('/').replace('dbo.','')}")

In [0]:
spark.sql(f"""CREATE DATABASE IF NOT EXISTS gold""")

DataFrame[]

In [0]:
%sql
create or replace table gold.dim_set as (
select s.set_num as set_id, s.name as set_name, s.year, s.num_parts, t.name as theme_name
from sets s
INNER JOIN themes t ON s.theme_id = t.id
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
create or replace table gold.dim_user as (
select *
from users
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE gold.dim_date (
  date_id INT PRIMARY KEY,
  date DATE NOT NULL,
  year INT NOT NULL,
  quarter INT NOT NULL,
  month INT NOT NULL,
  day INT NOT NULL,
  weekday INT NOT NULL,
  is_weekend INT NOT NULL COMMENT '1 for weekend, 0 for weekday'
);

In [0]:
from datetime import datetime, timedelta
from pyspark.sql.types import *

start_date = datetime(2024, 1, 1)
end_date = datetime(2026, 12, 31)

dates = []
current_date = start_date
date_id = 1

while current_date <= end_date:
    dates.append((
        date_id,
        current_date.date(),
        current_date.year,
        (current_date.month - 1) // 3 + 1,
        current_date.month,
        current_date.day,
        current_date.weekday(),
        1 if current_date.weekday() in [5, 6] else 0  # 5=Saturday, 6=Sunday
    ))
    current_date += timedelta(days=1)
    date_id += 1

schema = StructType([
    StructField("date_id", IntegerType(), False),
    StructField("date", DateType(), False),
    StructField("year", IntegerType(), False),
    StructField("quarter", IntegerType(), False),
    StructField("month", IntegerType(), False),
    StructField("day", IntegerType(), False),
    StructField("weekday",IntegerType(), False),
    StructField("is_weekend", IntegerType(), False)
])

df = spark.createDataFrame(dates, schema)

df.createOrReplaceTempView("dim_date")

In [0]:
%sql
insert into gold.dim_date select * from dim_date

num_affected_rows,num_inserted_rows
1095,1095


In [0]:
%sql
CREATE OR REPLACE TABLE gold.fact_orders AS (
  WITH order_details_with_date AS (
    SELECT od.*, dd.date_id
    FROM order_details od
    INNER JOIN dim_date dd ON od.order_date = dd.date
  ),
  orders_with_set AS (
    SELECT o.order_id, o.user_id, odwd.set_num AS set_id, odwd.date_id, odwd.quantity, odwd.price, s.city_ship
    FROM orders o
    INNER JOIN order_details_with_date odwd ON o.order_id = odwd.order_id
    INNER JOIN shipments s ON o.order_id = s.order_id
  )
  select order_id,user_id,set_id,date_id,price,quantity,city_ship from orders_with_set)

num_affected_rows,num_inserted_rows


In [0]:
%sql
select *
from gold.fact_orders

order_id,user_id,set_id,date_id,price,quantity,city_ship
ORDER-27D9FE,USER-D1197C,065-1,93,15.68,8,Houston
ORDER-D2902C,USER-1B9A94,21000-1,93,53.97,1,New York
ORDER-CFE157,USER-7E2207,5008162-1,93,75.27,7,Houston
ORDER-684537,USER-2993B6,10701-1,93,53.2,9,Houston
ORDER-684537,USER-2993B6,10701-1,93,53.2,9,Phoenix
ORDER-387E6A,USER-AC3AF3,7511-1,93,47.51,4,Chicago
ORDER-387E6A,USER-AC3AF3,7511-1,93,47.51,4,Houston
ORDER-387E6A,USER-AC3AF3,7511-1,93,47.51,4,Houston
ORDER-387E6A,USER-AC3AF3,7511-1,93,47.51,4,Chicago
ORDER-A91A40,USER-F97B28,5007392-1,93,62.38,4,Houston


In [0]:
%sql

-- Drop constraints if they exist
ALTER TABLE gold.fact_orders DROP CONSTRAINT IF EXISTS fk_fact_orders_dim_set;
ALTER TABLE gold.fact_orders DROP CONSTRAINT IF EXISTS fk_fact_orders_dim_user;
ALTER TABLE gold.fact_orders DROP CONSTRAINT IF EXISTS fk_fact_orders_dim_date;

-- Drop primary key constraints if they exist
ALTER TABLE gold.fact_orders DROP CONSTRAINT IF EXISTS pk_fact_orders;
ALTER TABLE gold.dim_user DROP CONSTRAINT IF EXISTS pk_dim_user;
ALTER TABLE gold.dim_set DROP CONSTRAINT IF EXISTS pk_dim_set;
ALTER TABLE gold.dim_date DROP CONSTRAINT IF EXISTS pk_dim_date;
ALTER TABLE gold.dim_date DROP CONSTRAINT IF EXISTS dim_date_pk;
-- Recreate constraints
ALTER TABLE gold.fact_orders ALTER COLUMN order_id SET NOT NULL;
ALTER TABLE gold.dim_date ALTER COLUMN date_id SET NOT NULL;
ALTER TABLE gold.dim_user ALTER COLUMN user_id SET NOT NULL;
ALTER TABLE gold.dim_set ALTER COLUMN set_id SET NOT NULL;

ALTER TABLE gold.fact_orders ADD CONSTRAINT pk_fact_orders PRIMARY KEY (order_id);
ALTER TABLE gold.dim_user  ADD CONSTRAINT pk_dim_user PRIMARY KEY (user_id);
ALTER TABLE gold.dim_set  ADD CONSTRAINT pk_dim_set PRIMARY KEY (set_id);
ALTER TABLE gold.dim_date  ADD CONSTRAINT pk_dim_date PRIMARY KEY (date_id);

ALTER TABLE gold.fact_orders  ADD CONSTRAINT fk_fact_orders_dim_date FOREIGN KEY (date_id) REFERENCES gold.dim_date(date_id);
ALTER TABLE gold.fact_orders ADD CONSTRAINT fk_fact_orders_dim_user FOREIGN KEY (user_id) REFERENCES gold.dim_user(user_id);
ALTER TABLE gold.fact_orders  ADD CONSTRAINT fk_fact_orders_dim_set FOREIGN KEY (set_id) REFERENCES gold.dim_set(set_id);


In [0]:
#config rápida
account_name = "datalakeprojectrebrickab"
container_name = "gold"
account_key = ""

mount_point = f"/mnt/{container_name}"

if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
        source = f"wasbs://{container_name}@{account_name}.blob.core.windows.net",
        mount_point = mount_point,
        extra_configs = {f"fs.azure.account.key.{account_name}.blob.core.windows.net": account_key}
    )
    print("Montaje realizado exitosamente")
else:
    print(f"El punto {mount_point} ya está montado")

Montaje realizado exitosamente


In [0]:
def save_table_to_adls(table_name):
    df = spark.table(f"gold.{table_name}")
    
    output_path = f"/mnt/gold/rebrickable/delta_tables_data_modelling/{table_name}"
    
    df.write \
      .mode("overwrite") \
      .format("delta") \
      .option("overwriteSchema", "true") \
      .save(output_path)


tables = ["dim_set", "dim_user", "dim_date","fact_orders"]

for table in tables:
    save_table_to_adls(table)